<a href="https://www.kaggle.com/code/averma111/pytroch-llm-science-exam?scriptVersionId=138026597" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [40]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from tqdm.notebook import tqdm
from transformers import AutoTokenizer , AutoModel
import torch
from IPython.display import IFrame

import warnings
warnings.filterwarnings('ignore')
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
 

/kaggle/input/kaggle-llm-science-exam/sample_submission.csv
/kaggle/input/kaggle-llm-science-exam/train.csv
/kaggle/input/kaggle-llm-science-exam/test.csv


In [41]:
class DataAcquisition:
    
    def __init__(self):
        pass
    
    def prepare_data(self):
        path = '/kaggle/input/kaggle-llm-science-exam/train.csv'
        df = pd.read_csv(
            path, 
            sep=',', 
            low_memory=False
        )
        
        X = df.copy()
        return X
    
    def prepare_data_test(self):
        path = '/kaggle/input/kaggle-llm-science-exam/test.csv'
        df = pd.read_csv(
            path, 
            sep=',', 
            low_memory=False
        )
        
        X = df.copy()
        return X
    
    
    def new_features(self,df):
        pass
    
    
    
    def preprocessing(self,df):
        for idx in tqdm(range(df.shape[0]) , total = df.shape[0]):
            df["answer"][idx] = df[df["answer"][idx]][idx]
        return df
        
    def preprocessing_test(self,df):
        for idx in tqdm(range(df.shape[0]) , total = df.shape[0]):
            df["answer"][idx] = df[df["answer"][idx]][idx]
        return df
    
        
acq = DataAcquisition()

train_df = acq.prepare_data()
test_df = acq.prepare_data_test()

In [42]:
train_df.head()

,id,prompt,A,B,C,D,E,answer
0,0,Which of the following statements accurately d...,MOND is a theory that reduces the observed mis...,MOND is a theory that increases the discrepanc...,MOND is a theory that explains the missing bar...,MOND is a theory that reduces the discrepancy ...,MOND is a theory that eliminates the observed ...,D
1,1,Which of the following is an accurate definiti...,Dynamic scaling refers to the evolution of sel...,Dynamic scaling refers to the non-evolution of...,Dynamic scaling refers to the evolution of sel...,Dynamic scaling refers to the non-evolution of...,Dynamic scaling refers to the evolution of sel...,A
2,2,Which of the following statements accurately d...,The triskeles symbol was reconstructed as a fe...,The triskeles symbol is a representation of th...,The triskeles symbol is a representation of a ...,The triskeles symbol represents three interloc...,The triskeles symbol is a representation of th...,A
3,3,What is the significance of regularization in ...,Regularizing the mass-energy of an electron wi...,Regularizing the mass-energy of an electron wi...,Regularizing the mass-energy of an electron wi...,Regularizing the mass-energy of an electron wi...,Regularizing the mass-energy of an electron wi...,C
4,4,Which of the following statements accurately d...,The angular spacing of features in the diffrac...,The angular spacing of features in the diffrac...,The angular spacing of features in the diffrac...,The angular spacing of features in the diffrac...,The angular spacing of features in the diffrac...,D


In [43]:
train_df=acq.preprocessing(train_df)
train_df.drop(["id" , "A" , "B" , "C" , "D" , "E"] , axis = 1 , inplace = True)
train_df.head()

  0%|          | 0/200 [00:00<?, ?it/s]

,prompt,answer
0,Which of the following statements accurately d...,MOND is a theory that reduces the discrepancy ...
1,Which of the following is an accurate definiti...,Dynamic scaling refers to the evolution of sel...
2,Which of the following statements accurately d...,The triskeles symbol was reconstructed as a fe...
3,What is the significance of regularization in ...,Regularizing the mass-energy of an electron wi...
4,Which of the following statements accurately d...,The angular spacing of features in the diffrac...


In [44]:
models_dict= {
    #'rbl':'roberta-large',
    #'abv2':'albert-base-v2',
    #'msd':'microsoft/deberta-base',
    #'ggl':'google/electra-medium-discriminator',
    #'fb':'facebook/bart-base'
}

In [45]:
def tokenizer():
    X = train_df.copy()
    tokenizer = AutoTokenizer.from_pretrained("roberta-base")
    model = AutoModel.from_pretrained("roberta-base")
    for idx in tqdm(range(X.shape[0])):
        X["prompt"][idx] = tokenizer(X["prompt"][idx] , 
                                       return_tensors = "pt")["input_ids"]
    
    with torch.no_grad():
        X["answer"][idx] = model(
                tokenizer(X["answer"][idx] , 
                return_tensors = "pt")["input_ids"])[0][0][0]
    torch.cuda.empty_cache()
    return X
    
    
X=tokenizer()

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


  0%|          | 0/200 [00:00<?, ?it/s]

In [46]:
class LLMNNBase(torch.nn.Module):
    
    def __init__(self,n_neurons):
        super(LLMNNBase, self).__init__()
        self.model_type = AutoModel.from_pretrained("roberta-base")

        self.n_neurons = n_neurons
        self.linear_1 = torch.nn.Linear(self.n_neurons , self.n_neurons)
    
    def forward(self, inputs):
        inputs = self.model(inputs)[0]
        inputs = torch.mean(inputs, axis=1)
        output = self.linear_1(inputs)

        return output
    
    
llmb = LLMNNBase(768)

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [47]:
optim = torch.optim.Adam(llmb.parameters())
criterion = torch.nn.MSELoss()
losses = []
for x , y in tqdm(zip(X["prompt"] , X["answer"]) , total = X.shape[0]):
    x = torch.tensor(x , dtype = torch.long).to("cuda")
    y = torch.tensor(y , dtype = torch.float32).to("cuda")
    x = x.reshape(shape = (1 , x.shape[0]))
    if x.shape[1] > 512: x = x[: , :512]
    pred = llmb(x)[0]
    loss_fun = criterion(pred , y)
    losses.append(loss_fun)
    torch.cuda.empty_cache()
    optim.step

  0%|          | 0/200 [00:00<?, ?it/s]

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:6                                                                                    │
│                                                                                                  │
│    3 losses = []                                                                                 │
│    4 for x , y in tqdm(zip(X["prompt"] , X["answer"]) , total = X.shape[0]):                     │
│    5 │   x = torch.tensor(x , dtype = torch.long).to("cuda")                                     │
│ ❱  6 │   y = torch.tensor(y , dtype = torch.float32).to("cuda")                                  │
│    7 │   x = x.reshape(shape = (1 , x.shape[0]))                                                 │
│    8 │   if x.shape[1] > 512: x = x[: , :512]                                                    │
│    9 │   pred = llmb(x)[0]                                                                       │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
TypeError: new(): invalid data type 'str'